In [1]:
import cv2
import imutils
import numpy
import pytesseract

from skimage.segmentation import clear_border

In [ ]:
class LicensePlateReader:
    
    
    def __init__(self, minAspectRatio=4, maxAspectRatio=5, debugMode=False):
        
        self.minAspectRatio = minAspectRatio
        self.maxAspectRatio = maxAspectRatio
        self.debugMode = debugMode
        
        
    def debug_imshow(self, title, image, waitKey=False):
        
        if self.debugMode:
            cv2.imshow(title, image)
            
            if waitKey:
                cv2.waitKey(0)
                
                
    def locate_license_plate(self, grayImage, keep=5):
        
        rectangleKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
        blackHat = cv2.morphologyEx(grayImage, cv2.MORPH_BLACKHAT, rectangleKernel)
        self.debug_imshow("Black Hat", blackHat)
        
        squareKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        morphCloseImage = cv2.morphologyEx(grayImage, cv2.MORPH_CLOSE, squareKernel)
        thresholdImage = cv2.threshold(morphCloseImage, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        self.debug_imshow("Black Hat", thresholdImage)
        
        gradientX = cv2.Sobel(blackHat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        gradientX = numpy.absolute(gradientX)
        
        minValue = numpy.min(gradientX)
        maxValue = numpy.max(gradientX)
        
        gradientX = 255 * ((gradientX - minValue) / (maxValue - minValue))
        gradientX = gradientX.asType("uint8")
        self.debug_imshow("Scharr", gradientX)
        
        gradientX = cv2.GaussianBlur(gradientX, (5, 5), 0)
        gradientX = cv2.morphologyEx(gradientX, cv2.MORPH_CLOSE, rectangleKernel)
        thresholdGradient = cv2.threshold(gradientX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        self.debug_imshow("Gradient Threshold", thresholdGradient)